#This is a project that analyzes candidate CVs and job descriptions, and returns the top 5 CVs per JD.

# PDF data extractor
Here we are prepping the CVs and extracting relevant data (skills, education, job role).

### Importing required libraries:

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber as plum
import re, spacy, glob, os

In [ ]:
!python -m spacy download en_core_web_sm

2023-09-19 11:13:52.365082: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Parsing through the PDFs and preprocessing the text:

In [ ]:
def extracting_text(pdf):
    text = ""

    for i in range(len(pdf.pages)):
        page = pdf.pages[i]
        text += page.extract_text()

    return text

In [ ]:
def preprocessing_text(text):

    text = text.replace("\n", " ").strip()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()

    return text

### Getting the job category of the CV:

In [ ]:
def get_category(text):

     return text.split("\n")[0]

### Getting the educational qualifications from the CV:

In [ ]:
def get_education(text):

    keywords = [
        "High School", "Certificate", "Associate", "Diploma", "High School Diploma", "GED", "Undergraduate", "UG", "PG"
        "B.A.", "BA", "B.S.", "BS", "B.Sc.", "BSc", "B.Engg", "B.Eng.", "BTech", "B.Tech", "Bachelor", "Graduate",
        "M.A.", "MA", "M.S.", "MS", "M.Sc.", "MSc", "M.Eng.", "MTech", "M.Tech", "MBA", "Master", "Postgraduate",
        "Ph.D.", "PhD", "Doctorate", "Doctor", "Doctor of Medicine", "Doctor of Science"
    ]

    edu = []

    for i in keywords:
        pattern = r"(?i)\b{}\b".format(re.escape(i))
        match = re.search(pattern, text)
        if match: edu.append(match.group())

    return edu

### Getting the skills from the CV:

In [78]:
nlp = spacy.load("en_core_web_sm")


def get_skills(text):
    skills = []

    for i in nlp(text):
        if "NN" in i.tag_: skills.append(i.text)

    return list(set(skills))

### Uploading the CV dataset as a ZIP file and then extracting the CVs:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# location/path of zip file in colab runtime storage
resume_folder = "/content/drive/MyDrive/resumes.zip"

if zipfile.is_zipfile(resume_folder):
    with zipfile.ZipFile(resume_folder, 'r') as zip_ref:
        zip_ref.extractall()
        path = os.getcwd() + "/" + os.path.splitext(os.path.basename(resume_folder))[0]
else:
    print("# ERROR: Please upload a valid ZIP file with resume PDFs.")

### Storing CV data embeddings:

In [ ]:
! pip install tqdm

In [ ]:
import tqdm

skillembeds = []
edembeds = []
jobroles = []

pdf_directory = "/content/resumes/"
# In the above variable, you have to paste the path of the extracted folder (PDF dataset folder) from Colab runtime storage

all_resumes = glob.glob(os.path.join(pdf_directory, "*.pdf"))

# Use tqdm to add a progress bar
for pdf in tqdm.tqdm(all_resumes):  # Use tqdm.tqdm instead of enumerate
    currPDF = plum.open(pdf)
    text = extracting_text(currPDF)
    text = preprocessing_text(text)

    skillembeds.append(get_embeddings(get_skills(text)))
    edembeds.append(get_embeddings(get_education(text)))
    jobroles.append(get_category(text))


100%|██████████| 2484/2484 [1:11:23<00:00,  1.72s/it]


# Job Description analyzer:
Here we are working with 15 job descriptions from the given Hugging Face dataset.

### Loading the dataset:

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
jd_dataset = load_dataset("jacob-hugging-face/job-descriptions")

### Saving the features of 15 job roles in a dictionary:

In [ ]:
jobfeatures = {
    "company_name" : [],
    "position" : [],
    "req_skills" : [],
    "req_edu" : []
}


# jobs range
jobs = 12


for i in tqdm.tqdm(range(jobs)):
    # appending company name
    jobfeatures["company_name"].append(jd_dataset["train"][i]["company_name"])

    # appending offered position
    jobfeatures["position"].append(jd_dataset["train"][i]["position_title"])


    model_response = eval(jd_dataset["train"][i]["model_response"])

    # appending required skills
    jobfeatures["req_skills"].append(model_response["Required Skills"])

    # appending required education
    jobfeatures["req_edu"].append(model_response["Educational Requirements"])



100%|██████████| 12/12 [00:00<00:00, 1105.32it/s]


### Tokenizing the skills required for a given job description:

In [ ]:
jd_skillembedsList = []
jd_EdEmbedsList = []
jd_positions = []

for i in tqdm.tqdm(range(jobs)):

    jd_skillembedsList.append(get_embeddings(jobfeatures["req_skills"][i]))

    jd_EdEmbedsList.append(get_embeddings(jobfeatures["req_edu"][i]))

    jd_positions.append(jobfeatures["position"][i])


100%|██████████| 12/12 [00:09<00:00,  1.26it/s]


# Creating embeddings using DistilBERT and calculating cosine similarity:
Now we will proceed to use DistilBERT to create the embeddings we require and then use them to calculate cosine similarities that will give us an idea of how relevant the given CV is to the job role applied for.

### Installing the Transformers library and importing libraries:

In [ ]:
!pip install transformers

In [ ]:
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Creating embeddings:

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


def get_embeddings(tokenArray):
    tokenArrayInput = " ".join(tokenArray)
    tokenArrayEncode = tokenizer(tokenArrayInput, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        tokenArrayEmbeds = model(**tokenArrayEncode).last_hidden_state.mean(dim=1).numpy()

    return tokenArrayEmbeds

### Cosine similarity calculation for Education and Skills:

In [ ]:
def cos_similarity_scores(jd_skillembedsList, jd_EdEmbedsList, cvSkillEmbeds, cvEdEmbeds):

    skillScore = cosine_similarity(jd_skillembedsList, cvSkillEmbeds)
    edScore = cosine_similarity(jd_EdEmbedsList, cvEdEmbeds)
    return (skillScore + edScore) / 2.0

### Cosine similarity calculation for Category (job role) matching:

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

def category_similarity_score(position, category):

    tfidf_matrix = tfidf_vectorizer.fit_transform([position, category])
    cosineScore = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    return cosineScore[0][0]

### Importing libraries:

In [ ]:
import zipfile, os

# Driver function for the project:
This section of the code brings together all the aforementioned functions to create a functional CV-to-JD matcher.

### main() function:

In [ ]:
def main():

    for i in tqdm.tqdm(range(jobs)):
        CV_and_score = []

        for j in range(len(all_resumes)):

            # calculating scores
            skill_edu_score = cos_similarity_scores(jd_skillembedsList[i], jd_EdEmbedsList[i], skillembeds[j], edembeds[j])
            pos_score = category_similarity_score(jd_positions[i], jobroles[j])
            final_score = (skill_edu_score + pos_score) / 2.0
            CV_and_score.append((final_score[0][0], os.path.basename(all_resumes[j])))

        CV_and_score.sort(key=lambda x: x[0], reverse=True)
        top_scores = [score for score, _ in CV_and_score[:5]]
        top_CVs = [filename for _, filename in CV_and_score[:5]]


        print(f'\nCompany: {jobfeatures["company_name"][i]} ({jd_positions[i]}): \nTop 5 CVs: {top_CVs} \nCorresponding Scores: {top_scores}\n')


Scores have been left as is, in the previous cell. They can of course be modified to suit your needs.

### Calling main() function

In [ ]:
if __name__ == "__main__" : main()

  8%|▊         | 1/12 [00:12<02:21, 12.82s/it]


Company: Google (Sales Specialist): 
Top 5 CVs: ['10289113.pdf', '34131484.pdf', '24767027.pdf', '12082377.pdf', '30608780.pdf'] 
Corresponding Scores: [0.33589116, 0.33202514, 0.32912737, 0.32709986, 0.32338458]



 17%|█▋        | 2/12 [00:25<02:08, 12.88s/it]


Company: Apple (Apple Solutions Consultant): 
Top 5 CVs: ['15535920.pdf', '38457612.pdf', '15119529.pdf', '64017585.pdf', '26291616.pdf'] 
Corresponding Scores: [0.3421532, 0.33897674, 0.33826032, 0.33602178, 0.3348015]



 25%|██▌       | 3/12 [00:39<02:00, 13.41s/it]

...


Company: General Dynamics Information Technology (SR. Web Designer): 
Top 5 CVs: ['29524570.pdf', '51018476.pdf', '78149576.pdf', '93828034.pdf', '69243180.pdf'] 
Corresponding Scores: [0.29126108, 0.2848642, 0.28183237, 0.27538842, 0.2751118]



 92%|█████████▏| 11/12 [02:31<00:13, 13.87s/it]


Company: Sony Music Entertainment (Web Developer): 
Top 5 CVs: ['43311839.pdf', '22351830.pdf', '93828034.pdf', '17823436.pdf', '35990852.pdf'] 
Corresponding Scores: [0.35823363, 0.3417488, 0.3402963, 0.3386415, 0.33855093]



100%|██████████| 12/12 [02:45<00:00, 13.75s/it]


Company: Snapshot Interactive (Web Developer): 
Top 5 CVs: ['43311839.pdf', '93828034.pdf', '29524570.pdf', '16186411.pdf', '36758947.pdf'] 
Corresponding Scores: [0.28815103, 0.2793807, 0.27754253, 0.27396056, 0.27330464]



Execution will begin from the above line of code.